In [0]:
# Define your parameters
storage_account_name = "hexastorage145"
container_name = "retailsales"
sas_token = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-07-14T18:40:45Z&st=2025-07-14T10:40:45Z&spr=https&sig=WHO6qQstYeEC43YEsxeKKDab4clnRRcH426C%2Bx2gQDg%3D"  # should start with "?sv="

# Mount path
mount_point = f"/mnt/{container_name}"

# Unmount if already mounted
if mount_point in [mnt.mountPoint for mnt in dbutils.fs.mounts()]:
    dbutils.fs.unmount(mount_point)

# Mount storage
dbutils.fs.mount(
  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
  mount_point = mount_point,
  extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token}
)

# Check mount
display(dbutils.fs.ls(mount_point))

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import input_file_name

# Define schema
schema = StructType([
    StructField("Store", IntegerType()),
    StructField("DayOfWeek", IntegerType()),
    StructField("Date", StringType()),
    StructField("Sales", DoubleType()),
    StructField("Customers", IntegerType()),
    StructField("Open", IntegerType()),
    StructField("Promo", IntegerType()),
    StructField("StateHoliday", StringType()),
    StructField("SchoolHoliday", IntegerType())
])

input_path = "/mnt/retailsales/raw/"
bronze_path = "/mnt/retailsales/bronze/train/"
checkpoint_path = "/mnt/retailsales/checkpoints/bronze/"

df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("cloudFiles.useNotifications", "false")
    .schema(schema)
    .load(input_path)
    .withColumn("source_file", input_file_name())
)

(df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .start(bronze_path)
)
